#PharmRL - Pharmacophore elucidation with Deep Geometric Reinforcement Learning

## Generate Pharmacophores targeting a receptor with reinforcement learning.

### Restart session if you want to change entered values

#### Use a GPU backend

In [ ]:
#@title Clone github repository
%%capture
!pip install crcmod
!git clone --recurse-submodules https://github.com/RishalAggarwal/Pharmrl.git
%cd Pharmrl

In [ ]:
#@title Download dependencies - takes about 1-2 mins
!gsutil cp gs://koes-data/pharmnnrl-env.tar /content/

Copying gs://koes-data/pharmnnrl-env.tar...
/ [1 files][  6.9 GiB/  6.9 GiB]   18.1 MiB/s                                   
Operation completed over 1 objects/6.9 GiB.                                      


In [ ]:
#@title Setup environment - takes about 1 min to run
%%capture
!tar -xf /content/pharmnnrl-env.tar -C /content/
!pip install py3Dmol
OLD_LD_LIBRARY_PATH = %env LD_LIBRARY_PATH
%env LD_LIBRARY_PATH=/content/temp-env/lib/:$OLD_LD_LIBRARY_PATH
OLD_ENV_PYTHONPATH = %env PYTHONPATH
%env PYTHONPATH=/content/temp-env/lib/python3.10/site-packages/:$OLD_ENV_PYTHONPATH
%env BABEL_LIBDIR=/content/temp-env/lib/openbabel/3.1.0/

import sys
sys.path.insert(0,'/content/temp-env/lib/python3.10/site-packages/')
import openbabel
from openbabel import pybel
import os
import json
from google.colab import files
import json_stream
import py3Dmol
import sys




def upload():
  file_upload=files._upload_files(multiple=False)
  file_name=list(file_upload.keys())[0]
  file_write=open(file_name,'ab')
  file_write.write(file_upload[file_name])
  file_write.close()
  return file_name

receptor_present=False
json_present=False
ligand_present=False
starter_json='None'
min_features='4'
max_features='10'

In [ ]:
#@title Upload target receptor - you can either upload a molecule file in standard formats or a pharmit session file (Json)
# @markdown (Run cell to enable upload)

receptor_file_type = 'molecule file' # @param ["molecule file", "json"]
if receptor_file_type=='molecule file':
  print('Upload Receptor')
  receptor=upload()
  receptor_present=True
  json_present=False
else:
  print('Upload Json')
  json_file=upload()
  json_file_suffix=json_file.split('.')[-1]
  if json_file_suffix=='json':
    with open(json_file,'r') as f:
      json_content=json_stream.load(f)
      try:
        receptor_content=json_content['receptor']
        receptor_present=True
        json_present=True
      except:
        raise ValueError('provided Json does not have receptor information')
  else:
    raise ValueError('uploaded file is not a Json')





Upload Receptor


In [ ]:
#@title Define binding site, this can be defined in one of three ways:
#@markdown - Having a ligand present in the previously uploaded pharmit session file (Json)
#@markdown - Uploading a ligand file
#@markdown - Defining the coordinates and size of the binding site directly

binding_site_input = 'ligand' # @param ["ligand", "json", "coordinates"]
#@markdown - If choosing ligand or json, specify how much you want to increase the size of the binding site beyond the ligand below:

autobox_extend = "4.0" # @param{type:"string"}

if binding_site_input=='ligand':
  print('Upload Ligand')
  ligand=upload()
  ligand_present=True
elif binding_site_input=='json':
  if not json_present:
    raise ValueError('Json not uploaded in previous step')
  with open(json_file,'r') as f:
      json_content=json_stream.load(f)
      try:
        ligand_content=json_content['ligand']
        ligand_present=True
      except:
        raise ValueError('provided Json does not have ligand information')

elif binding_site_input=="coordinates":
  ligand_present=False
#@markdown - Only fill in below fields if you have selected coordinates (floats)
  coord_x = "-15" # @param {type:"string"}
  coord_y = "15" # @param {type:"string"}
  coord_z = "45" # @param {type:"string"}
  size_x = "4.0" # @param {type:"string"}
  size_y = "4.0" # @param {type:"string"}
  size_z = "4.0" # @param {type:"string"}

Upload Ligand


In [ ]:
#@title Specify which Pharmacophore Features to use:
#@markdown - ligand_only - Use only features identified from the receptor-ligand complex
#@markdown - cnn_only - Use only features predicted by the CNN
#@markdown - combine - Use both ligand and CNN features

features_input = 'cnn_only' # @param ["ligand_only", "cnn_only", "combine"]
if features_input =='ligand_only' or features_input== 'combine':
  if not ligand_present:
    raise ValueError('ligand not provided above')


In [ ]:
#@title Starter Pharmacophore: run this cell if you have specific features that should be present in the generated pharmacophore
print('Upload starter json')
starter_json=upload()
starter_json_file_suffix=starter_json.split('.')[-1]
#@markdown To create a starter pharmacophore: upload features to pharmit -> enable the ones you like -> save session -> upload Json here
if not starter_json_file_suffix=='json':
  raise ValueError('Uploaded file is not a Json')

Upload starter json


In [ ]:
#@title Specify the min and max size of the generated pharmacophores (integers)
min_features = "3" # @param {type:"string"}
max_features = "6" # @param {type:"string"}

In [ ]:
#@title Generate Pharmacophores 1-2 mins
!chmod -R 777 /content/temp-env/bin/python
if binding_site_input=='coordinates':
  if json_present:
    !/content/temp-env/bin/python predict.py --input_json {json_file} --starter_json {starter_json} --x_center {coord_x} --y_center {coord_y} --z_center {coord_z} --x_size {size_x} --y_size {size_y} --z_size {size_z} --features {features_input} --min_features {min_features} --max_features {max_features}
  else:
    !/content/temp-env/bin/python predict.py --receptor {receptor} --starter_json {starter_json} --x_center {coord_x} --y_center {coord_y} --z_center {coord_z} --x_size {size_x} --y_size {size_y} --z_size {size_z} --features {features_input} --min_features {min_features} --max_features {max_features}
elif binding_site_input=='ligand':
  if json_present:
    !/content/temp-env/bin/python predict.py --input_json {json_file} --starter_json {starter_json} --ligand {ligand} --features {features_input} --min_features {min_features} --max_features {max_features}
  else:
    !/content/temp-env/bin/python predict.py --receptor {receptor} --starter_json {starter_json} --ligand {ligand} --features {features_input} --min_features {min_features} --max_features {max_features}
else:
  !/content/temp-env/bin/python predict.py --input_json {json_file} --starter_json {starter_json} --features {features_input} --min_features {min_features} --max_features {max_features}

Streaming output truncated to the last 5000 lines.
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2162)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2163)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2164)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element symbol of an atom.
  but OpenBabel found '  ' (atom 2165)
*** Open Babel Warning  in parseAtomRecord
  Problems reading a HETATM or ATOM record.
  According to the PDB specification,
  columns 77-78 should contain the element sy

In [ ]:
#@title Visualize Generated Pharmacophores
# @markdown CMap: Aromatic: Purple, HydrogenDonor: white, HydrogenAcceptor: Orange, NegativeIon: Red, PositiveIon: Blue
res = json.load(open('pharmnn_predicted_pharmacophores.json','r'))


cmap  = {'Aromatic': 'purple',
         'HydrogenDonor': '0xf0f0f0',
         'HydrogenAcceptor': 'orange',
         'Hydrophobic': 'green',
         'NegativeIon': 'red',
         'PositiveIon': 'blue',
         'ExclusionSphere': 'grey',
        'InclusionSphere': 'yellow'};

model_names = sorted({p['label'] for p in res['points']})
v = py3Dmol.view(width="100%", height="400px",viewergrid=(1,len(model_names)))
if ligand_present:
  v.addModel(res['ligand'])
  v.setStyle({'model':0},{'stick':{'colorscheme':'Jmol'}})
else:
  v.center({'x':coord_x,'y':coord_y,'z':coord_z})

for i,mname in enumerate(model_names):
  v.addLabel(f"Model {i}",{'position':{'x':0,'y':0,'z':0},'useScreen': True, 'fontSize': 24},viewer=(0,i));
  for p in res['points']:
    if p['label'] == mname:
      radius=p['radius']
      if p['name']=='Hydrophobic' or p['name'] == 'HydrogenDonor':
        radius=p['radius']*0.9
      elif 'ion' in p['name']:
        radius=p['radius']*0.8
      v.addSphere({'center':p,'radius':radius,
                   'color':cmap[p['name']],
                   'wireframe': True,
                    'linewidth': 1.5},
                  viewer=(0,i))
v.zoomTo()
v.addModel(res['receptor'])
v.addSurface('VDW', { 'opacity': 0.8, 'colorscheme': 'whiteCarbon' })
v.setStyle({'model':-1},{'cartoon':{},'stick':{'radius':0.15}})

v.show()

3Dmol.js failed to load for some reason. Please check your browser console for error messages.

In [ ]:
#@title Load Pharmacophores into Pharmit (switch off pop up blocker)
predicted_json=open('pharmnn_predicted_pharmacophores.json','r').readlines()

from google.colab import output

javascript_code="""
//object for sending messages to a window, but only after we receive an ack
function Message(data, w, dest) {
    var curWindow = w;
    var curDest = dest;
    var curMsg = data;
    var isAcked = 0;

    function receiveMessage(event) {
        if (event.data == "ack2") {
            isAcked = 1;
        }
    }

    function check() {
        if (isAcked) {
            curWindow.postMessage(curMsg, curDest);
            curDest = "";
            curMsg = "";
            curWindow = null;
            isAcked = 0;
            window.removeEventListener("message", receiveMessage);
        } else if (curWindow) {
            curWindow.postMessage("ack", curDest);
            setTimeout(check, 250);
        }
    }

    window.addEventListener("message", receiveMessage);
    w.postMessage("ack", dest);
    setTimeout(check, 250);
}
var win = window.open("http://pharmit.csb.pitt.edu/search.html");
var msg = new Message(JSON.stringify("""+predicted_json[0]+"""), win, '*');
"""
output.eval_js(javascript_code)




Facing Problems? Raise an issue at: https://github.com/RishalAggarwal/Pharmrl

\\


If you found this useful, please cite:

